# Using the attribution runs to look at tropical Atlantic variability

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
def load_cvdp_timeseries(ts_name,dir_name):
    nc_file_name=ts_name+"."+dir_name+".nc"
    if os.path.exists(nc_file_name):
        # Load the dataset using xarray
        da = xr.open_dataset(nc_file_name)
        print('Loaded '+ts_name+' from pre-existing file called '+nc_file_name)
    else:
        print('Extracting '+ts_name+' from all the cvdp_files in '+dir_name+' ...')
        # Look in the provided directory
        file_names = os.listdir('cmip6.hist-aer.cvdp_data')
        # Filter the file names to include only those with the .nc extension
        paths = [file_name for file_name in file_names if file_name.endswith('.nc')]
        #Load in an example file to get some metadata
        dataset = xr.open_dataset(dir_name+'/'+paths[0],decode_times=False)
        dataset.time.attrs['calendar']='360_day'
        dataset.time_mon1.attrs['calendar']='360_day'
        dataset.time_mon2.attrs['calendar']='360_day'
        dataset.time_mon3.attrs['calendar']='360_day'
        dataset=xr.decode_cf(dataset,decode_times=True)
        dataset=dataset.convert_calendar('proleptic_gregorian',align_on='year')
        time=dataset.time
        num_months = time.size #number of months (this code assumes all files have equal length!)
        # Number of ensemble members
        num_ensemble_members = len(paths)
        # Create a DataArray to hold all the ensemble members
        ensemble_member = xr.IndexVariable('ensemble_member', np.arange(num_ensemble_members))
        da = xr.DataArray(np.full((num_ensemble_members,num_months),np.nan), coords=[ensemble_member,time],
                          dims=['ensemble_member', 'time'], name=ts_name)
        # Fill up the dataArray by looping over each file
        for i in range(1,num_ensemble_members-1):
            # Open the dataset using xarray
            dataset = xr.open_dataset(dir_name+'/'+paths[i],decode_times=False)
            dataset.time.attrs['calendar']='360_day'
            dataset.time_mon1.attrs['calendar']='360_day'
            dataset.time_mon2.attrs['calendar']='360_day'
            dataset.time_mon3.attrs['calendar']='360_day'
            dataset=xr.decode_cf(dataset,decode_times=True)
            dataset=dataset.convert_calendar('proleptic_gregorian',align_on='year')
            # Extract time series of interest
            da[i,:] = dataset[ts_name]

        # Print the information about the DataArray
        print('and saved it to a file called '+nc_file_name+'for later use')
        da.to_netcdf(nc_file_name)
    
    return da

In [ ]:
aer_array=load_cvdp_timeseries('atlantic_nino','cmip6.hist-aer.cvdp_data')
ghg_array=load_cvdp_timeseries('atlantic_nino','cmip6.hist-GHG.cvdp_data')
ghg_array


In [ ]:
# convert to anomalies
aer_array.plot()
plt.title('ATL3 in aerosol-driven runs (oC)')

In [ ]:
# Compute the running standard deviation
window_size = 120  # Number of months in the running window
running_std = aer_array.rolling(time=window_size).std()
# convert to anomalies
running_std=running_std-running_std.sel(time=slice('1850-01-01','1899-12-31')).mean(dim='time')
running_std.plot()
plt.title('Aerosol-driven change in ATL3 std dev. (since 1850-99, oC^2)')

In [ ]:
#Create some plots
#mean_state=ghg_array-ghg_array.sel(time=slice('1850-01-01','1899-12-31')).mean(dim='time')
mean_state=ghg_array
mean_state.plot()
plt.title('ATL3 in GHG-driven runs (oC)')

In [ ]:
# Compute the running standard deviation
window_size = 120  # Number of months in the running window
running_std = ghg_array.rolling(time=window_size).std()
# convert to anomalies
running_std=running_std-running_std.sel(time=slice('1850-01-01','1899-12-31')).mean(dim='time')
running_std.plot()
plt.title('GHG-driven change in ATL3 std dev. (since 1850-99, oC^2)')

#### Now create a more conventional time series figure. Basing this on IPCC AR6 WG1 Fig 3.9 

In [ ]:
# Compute the median and percentile ranges along the y-axis
print(aer_array)
median = aer_array.median(dim='ensemble_member')
percentile_10 = aer_array.quantile(0.1, dim='ensemble_member')
percentile_90 = aer_array.quantile(0.9, dim='ensemble_member')
print(median)

# Plot the results
plt.plot(aer_array.time, median.atlantic_nino, label='Median')
plt.fill_between(aer_array.time, percentile_10, percentile_90, alpha=0.2, label='10th-90th Percentile Range')

# Customize the plot
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Median and Percentile Range')
plt.legend()

# Show the plot
plt.show()